In [162]:
import pandas as pd
import numpy as np
import torch
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import xgboost as xgb


In [163]:
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']

In [164]:
def criterion(prds, tgts, weight=None):
    return (torch.sqrt(torch.nn.MSELoss()(prds[:,0], tgts[:,0])) +
            torch.sqrt(torch.nn.MSELoss()(prds[:,1], tgts[:,1])) +
            torch.sqrt(torch.nn.MSELoss()(prds[:,2], tgts[:,2])) + 
            torch.sqrt(torch.nn.MSELoss()(prds[:,3], tgts[:,3])) + 
            torch.sqrt(torch.nn.MSELoss()(prds[:,4], tgts[:,4]))) / 5.0

In [165]:
def sort_df(df, train_df):
    tmp_df = pd.merge(train_df['id_seqpos'], df[['id_seqpos'] + pred_cols], on='id_seqpos')
    return tmp_df

# Load Valid

In [166]:
valid_cnnrnn = pd.read_csv('../OpenVaccine/valid/validation_cnnrnn.csv')
valid_cnnrnn_2 = pd.read_csv('../OpenVaccine/valid/validation_cnnrnn_groupkfold.csv')

valid_deepgcn = pd.read_csv('../OpenVaccine/valid/validation_deepgcn.csv')
valid_deepgcn_2 = pd.read_csv('../OpenVaccine/valid/validation_deepgcn_khanh_1.csv')
valid_deepgcn_3 = pd.read_csv('../OpenVaccine/valid/validation_deepgcn_khanh_2.csv')
valid_deepgcn_4 = pd.read_csv('../OpenVaccine/valid/validation_deepgcn_pseudo.csv')

valid_ae = pd.read_csv('../OpenVaccine/valid/validation_aepretrain.csv')
valid_ae_2 = pd.read_csv('../OpenVaccine/valid/validation_aepretrain_2.csv')
valid_ae_3 = pd.read_csv('../OpenVaccine/valid/validation_aepretrain_pytorch.csv')
valid_ae_4 = pd.read_csv('../OpenVaccine/valid/validation_aepytorch_noweight_pseudo.csv')
valid_ae_5 = pd.read_csv('../OpenVaccine/valid/validation_aepytorch_pseudo.csv')
valid_ae_6 = pd.read_csv('../OpenVaccine/valid/validation_aepytorch_noweight_pseudo_150e.csv')


# valid_list = [valid_cnnrnn, 
#               valid_cnnrnn_2, 
#               valid_deepgcn, 
#               valid_deepgcn_2, 
#               valid_deepgcn_3, 
#               valid_ae,
#               valid_ae_2,
#               valid_ae_3
#              ]

valid_list = [
              valid_ae_4,
              valid_ae_5,
              valid_ae_6
             ]

# Load Train

In [167]:
train = pd.read_json("../OpenVaccine/train.json", lines=True)
train = train[((train['SN_filter'] == 1) & (train['signal_to_noise'] > 1))]

In [168]:
train.head()

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
5,5,id_00ab2d761,GGAAAGCGCCGCGGCGGUAGCGGCAGCGAGGAGCGCUACCAAGGCA...,.....(.(((((.(((((((((...........)))))))..(((....,EEEEESISSSSSISSSSSSSSSHHHHHHHHHHHSSSSSSSMMSSSH...,4.136,1,107,68,"[0.1942, 0.2041, 0.1626, 0.1213, 0.10590000000...","[0.2726, 0.2984, 0.21660000000000001, 0.1637, ...","[0.3393, 0.2728, 0.2005, 0.1703, 0.1495, 0.134...","[0.165, 0.20520000000000002, 0.179, 0.1333, 0....","[0.2864, 0.24710000000000001, 0.2222, 0.1903, ...","[0.7642, 1.6641, 1.0622, 0.5008, 0.4107, 0.133...","[0.9559000000000001, 1.9442, 1.0114, 0.5105000...","[1.9554, 2.1298, 1.0403, 0.609, 0.5486, 0.386,...","[0.22460000000000002, 1.7281, 1.381, 0.6623, 0...","[0.5882000000000001, 1.1786, 0.9704, 0.6035, 0..."
6,6,id_00abef1d7,GGAAAACAAUUGCAUCGUUAGUACGACUCCACAGCGUAAGCUGUGG...,.........((((((((......((((((((((((....)))))))...,EEEEEEEEESSSSSSSSIIIIIISSSSSSSSSSSSHHHHSSSSSSS...,2.485,1,107,68,"[0.422, 0.5478000000000001, 0.4749000000000000...","[0.4801, 0.7943, 0.42160000000000003, 0.397300...","[0.9822000000000001, 1.272, 0.6940000000000001...","[0.5827, 0.7555000000000001, 0.5949, 0.4511, 0...","[0.9306000000000001, 1.0496, 0.5844, 0.7796000...","[0.895, 2.3377, 2.2305, 2.003, 1.9006, 1.0373,...","[0.46040000000000003, 3.6695, 0.78550000000000...","[2.7711, 7.365, 1.6924000000000001, 1.43840000...","[1.073, 2.8604000000000003, 1.9936, 1.0273, 1....","[2.0964, 3.3688000000000002, 0.6399, 2.1053, 1..."
7,7,id_00b436dec,GGAAAUCAUCGAGGACGGGUCCGUUCAGCACGCGAAAGCGUCGUGA...,.....(((((((((((..(((((((((..((((....))))..)))...,EEEEESSSSSSSSSSSIISSSSSSSSSIISSSSHHHHSSSSIISSS...,1.727,1,107,68,"[0.4843, 0.5233, 0.4554, 0.43520000000000003, ...","[0.8719, 1.0307, 0.6649, 0.34500000000000003, ...","[0.7045, 0.7775000000000001, 0.5662, 0.4561, 0...","[0.384, 0.723, 0.4766, 0.30260000000000004, 0....","[0.7429, 0.9137000000000001, 0.480400000000000...","[1.1576, 1.5137, 1.3382, 1.5622, 1.2121, 0.295...","[1.6912, 5.2652, 2.3901, 0.45890000000000003, ...","[1.8641, 2.3767, 1.149, 1.0132, 0.9876, 0.0, 0...","[0.49060000000000004, 4.6339, 1.95860000000000...","[1.2852000000000001, 2.5460000000000003, 0.234..."


In [169]:
train_ls = []
for i, row in train.iterrows():
    single_df = pd.DataFrame(np.vstack(row[pred_cols].apply(lambda x: np.array(x)).to_numpy()).T, columns=pred_cols)
    id = row['id']
    single_df['id_seqpos'] = [f'{id}_{x}' for x in range(single_df.shape[0])]
    
    train_ls.append(single_df)
    
train_df = pd.concat(train_ls)

In [170]:
train_df.head()

,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C,id_seqpos
0,0.3297,0.7556,2.3375,0.3581,0.6382,id_001f94081_0
1,1.5693,2.9830,3.5060,2.9683,3.4773,id_001f94081_1
2,1.1227,0.2526,0.3008,0.2589,0.9988,id_001f94081_2
3,0.8686,1.3789,1.0108,1.4552,1.3228,id_001f94081_3
4,0.7217,0.6376,0.2635,0.7244,0.7877,id_001f94081_4


# Validate

In [171]:
for i, valid_df in enumerate(valid_list):
    valid_df = valid_df.groupby('id_seqpos')[pred_cols].mean().reset_index()
    valid_list[i] = sort_df(valid_df, train_df)

In [172]:
for valid_df in valid_list:
    valid_score = criterion(torch.tensor(valid_df[pred_cols].to_numpy()), torch.tensor(train_df[pred_cols].to_numpy()))
    print(f'valid_score: {valid_score}')

valid_score: 0.19885673776694818
valid_score: 0.21206127808069414
valid_score: 0.19728948682212716


# Blending

In [82]:
def blend_score(weights):
    weights = weights/weights.sum()
    
    valid = None
    for i, valid_df in enumerate(valid_list):
        if valid is None:
            valid = valid_df[pred_cols].to_numpy() * weights[i]
        else:
            valid += valid_df[pred_cols].to_numpy() * weights[i]
   
    return criterion(torch.tensor(valid), torch.tensor(train_df[pred_cols].to_numpy()))    

## Average

In [83]:
blend_score(np.ones((len(valid_list)))) #tensor(0.1995, dtype=torch.float64)

tensor(0.2030, dtype=torch.float64)

## Minimize

In [84]:
lls = []
wghts = []

for i in range(10):
    starting_values = np.random.uniform(size=len(valid_list))
    print(f'#Step: {i} Starting Values: {starting_values}')
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
    bounds = [(0, 1)] * len(valid_list)

    res = minimize(blend_score, starting_values, method='COBYLA', bounds=bounds, 
                   options={'disp': True, 'maxiter': 1000})
    
    lls.append(res['fun'])
    wghts.append(res['x'])
    print('Weights: {weights}  Score: {score}'.format(weights=res['x'], score=res['fun']))

bestSC = np.min(lls)
bestWght = wghts[np.argmin(lls)]

print('\n Ensemble Score: {best_score}'.format(best_score=bestSC))
print('\n Best Weights: {weights}'.format(weights=bestWght))

#Step: 0 Starting Values: [0.29263239 0.71643285]


/home/thinh/anaconda3/lib/python3.7/site-packages/scipy/optimize/_minimize.py:540: RuntimeWarning: Method COBYLA cannot handle bounds.
  RuntimeWarning)


Weights: [1.73826043 1.88522798]  Score: 0.20295290295498983
#Step: 1 Starting Values: [0.276885   0.22879901]
Weights: [0.16071268 0.17416087]  Score: 0.2029529050800408
#Step: 2 Starting Values: [0.93227879 0.48661992]
Weights: [1.41649767 1.53654867]  Score: 0.2029529029550546
#Step: 3 Starting Values: [0.14237003 0.01340158]
Weights: [1.00437605 1.08946513]  Score: 0.2029529029420642
#Step: 4 Starting Values: [0.12479891 0.50113984]
Weights: [2.00595743 2.17564621]  Score: 0.20295290293912358
#Step: 5 Starting Values: [0.1339376  0.54820776]
Weights: [1.9438724  2.10851068]  Score: 0.2029529029362888
#Step: 6 Starting Values: [0.10046533 0.20738611]
Weights: [-0.53882349 -0.58434288]  Score: 0.2029529029980047
#Step: 7 Starting Values: [0.67710634 0.46431947]
Weights: [0.6535316  0.70892818]  Score: 0.20295290296143276
#Step: 8 Starting Values: [0.80263978 0.79733736]
Weights: [0.68471686 0.74254005]  Score: 0.20295290302421173
#Step: 9 Starting Values: [0.17672573 0.93085425]
Weig

In [15]:
# Ensemble Score: 0.2018889359109685
# Ensemble Score: 0.19793112584421507
# Ensemble Score: 0.1972453394456028
# Ensemble Score: 0.1971927625696471


SyntaxError: invalid syntax (<ipython-input-15-7d1c990b89e6>, line 1)

## Markov Chain Monte Carlo

In [16]:
n = 1000
counter = 0
result = {}
num = len(valid_list)
weight = np.array([1.0/num,]*num)
old_score = 10.0
best_score = 10.0
best_weight = None

for i in range(0, n):
    new_weights = weight + np.array([0.005,] * num) * np.random.normal(loc=0.0, scale=1.0, size=num)
    new_weights[new_weights < 0.01]=0.01

    new_score = blend_score(new_weights)
#     print(new_score)
    if best_score > new_score:
        best_score = new_score
        best_weight = new_weights
        
    diff = new_score - old_score
    prob = min(1, np.exp(-diff/.3))
    random_prob = np.random.rand()
    
    if random_prob < prob:
        result[i] = (new_score, old_score, prob, random_prob, new_weights)
        weight = new_weights
        old_score = new_score
        counter += 1

print(f'Best score {best_score}')
print(f'Best weights {best_weight}')

Best score 0.19875703793503577
Best weights [0.14286159 0.2379039  0.31654853 0.28240266 0.1834408 ]


# Stacking

In [17]:
col = 'reactivity'

In [18]:
X = np.array([valid_df[col].to_numpy() for valid_df in valid_list]).T
y = train_df[col].to_numpy()

In [19]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X,y)
y_pred = xg_reg.predict(X)

[17:50:12] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:50:12] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [20]:
torch.sqrt(torch.nn.MSELoss()(torch.tensor(y_pred), torch.tensor(y)))

tensor(0.2451, dtype=torch.float64)

In [21]:
print("Mean:", torch.sqrt(torch.nn.MSELoss()(torch.tensor(X.mean(axis=1)), torch.tensor(y))))

Mean: tensor(0.1837, dtype=torch.float64)


In [22]:
col = 'deg_Mg_pH10'
X = np.array([valid_df[col].to_numpy() for valid_df in valid_list]).T
y = train_df[col].to_numpy()

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X,y)
y_pred = xg_reg.predict(X)

print(torch.sqrt(torch.nn.MSELoss()(torch.tensor(y_pred), torch.tensor(y))))
print("Mean:", torch.sqrt(torch.nn.MSELoss()(torch.tensor(X.mean(axis=1)), torch.tensor(y))))

[17:50:12] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:50:12] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
tensor(0.2885, dtype=torch.float64)
Mean: tensor(0.2287, dtype=torch.float64)
